In [35]:
import libs.fs
import parse
import numpy as np
import pandas as pd
import io
import scipy.interpolate
import matplotlib.pyplot as plt
import datetime

In [16]:
TEMPLATE = """RepRapFirmware height map file v2 generated at {datetime}, min error {min:.3f}, max error {max:.3f}, mean {mean:.3f}, deviation {std:.3f}
axis0,axis1,min0,max0,min1,max1,radius,spacing0,spacing1,num0,num1
X,Y,-170.00,169.00,-170.00,170.00,-1.00,56.49,56.66,7,7
{data}
"""

In [28]:
PATH = "../duet2/sys/heightmaps/"
filenames = [
    "heightmap_020C.csv",
    "heightmap_040C.csv",
    "heightmap_060C.csv",
    "heightmap_080C.csv",
]

In [41]:
!cat ../duet2/sys/heightmaps/heightmap_020C.csv

RepRapFirmware height map file v2 generated at 2022-11-17 23:24, min error -0.206, max error 0.130, mean -0.022, deviation 0.090
axis0,axis1,min0,max0,min1,max1,radius,spacing0,spacing1,num0,num1
X,Y,-170.00,169.00,-170.00,170.00,-1.00,56.49,56.66,7,7
 -0.056, -0.111, -0.193, -0.175,  0.005,  0.015,  0.061
  0.130,  0.015, -0.088, -0.099, -0.014,  0.106,  0.089
  0.125,  0.007, -0.121, -0.170, -0.094,  0.045,  0.039
  0.086, -0.019, -0.149, -0.206, -0.110,  0.045,  0.037
  0.082,  0.011, -0.101, -0.135, -0.078,  0.100,  0.075
  0.057, -0.043, -0.081, -0.094, -0.005,  0.084,  0.070
  0.015, -0.063, -0.114, -0.079, -0.025,  0.035,  0.020


In [29]:
def load(filename):
    with open(PATH + filename) as f:
        content = f.read()
    parsed = parse.parse(TEMPLATE, content)
    temp = int(filename[-8:-5])
    values = pd.read_csv(io.StringIO(parsed["data"]), header=None).values
    return temp, values

In [30]:
all_temps = [load(filename) for filename in filenames]

In [32]:
temps, values = zip(*all_temps)
temps = np.array(temps)
values = np.array(values)
print(temps.shape, values.shape)
interp = scipy.interpolate.interp1d(temps, values, axis=0, fill_value="extrapolate")

(4,) (4, 7, 7)


In [44]:
DATETIME = datetime.datetime.now().strftime("%Y-%m-%d %H:%M")

def write_interp(t):
    y = interp(t)
    data_rows = [
        "".join(f"  {val:.3f}," for val in row)[:-1]
        for row in y
    ]
    data_csv = "\n".join(data_rows)
    content = TEMPLATE.format(
        min=np.min(y),
        max=np.max(y),
        mean=np.mean(y),
        std=np.std(y),
        data=data_csv,
        datetime=DATETIME
    )
    with open(PATH + "heightmap_{t}C_interp.csv".format(t=str(t).zfill(3)), "wt") as f:
        f.write(content)

In [45]:
for t in [20, 30, 40, 50, 60, 70, 80]:
    write_interp(t)

In [47]:
!tree -a ..

..
├── .git
│   ├── COMMIT_EDITMSG
│   ├── HEAD
│   ├── config
│   ├── description
│   ├── hooks
│   │   ├── applypatch-msg.sample
│   │   ├── commit-msg.sample
│   │   ├── fsmonitor-watchman.sample
│   │   ├── post-update.sample
│   │   ├── pre-applypatch.sample
│   │   ├── pre-commit.sample
│   │   ├── pre-merge-commit.sample
│   │   ├── pre-push.sample
│   │   ├── pre-rebase.sample
│   │   ├── pre-receive.sample
│   │   ├── prepare-commit-msg.sample
│   │   ├── push-to-checkout.sample
│   │   └── update.sample
│   ├── index
│   ├── info
│   │   └── exclude
│   ├── logs
│   │   ├── HEAD
│   │   └── refs
│   │       ├── heads
│   │       │   └── main
│   │       └── remotes
│   │           └── origin
│   │               └── main
│   ├── objects
│   │   ├── 01
│   │   │   └── 4a7f641d6cc69c55bf93e882474510a6f63363
│   │   ├── 02
│   │   │   ├── cffe43c65ba2a4f1af70cb9c550c4ff789040f
│   │   │   └── e3f27da690a49ad299a6c2afd002ca4e8f6b8e
│   │   ├── 04
│   │   │   └── a6bb967b35316a301f